# 7.

In [2]:
from sklearn.datasets import make_moons

X_moons, y_moons = make_moons(n_samples=10000, noise=0.4, random_state=42)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_moons, y_moons, test_size=0.2, random_state=42)

In [4]:
from sklearn.tree import DecisionTreeClassifier

tree_reg = DecisionTreeClassifier(max_depth=10, random_state=42)
tree_reg.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=10, random_state=42)

In [5]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, X_train, y_train,cv=5)

In [6]:
scores

array([0.826875, 0.84125 , 0.843125, 0.840625, 0.843125])

In [7]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 6]
}

grid_search = GridSearchCV(tree_reg, param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

In [8]:
dicision_tree_best = grid_search.best_estimator_

In [9]:
scores = cross_val_score(dicision_tree_best, X_train, y_train, cv=5)

In [10]:
scores.mean()

0.8558749999999999

# 8.

In [11]:
from sklearn.model_selection import ShuffleSplit

shuffle_split = ShuffleSplit(n_splits=1000, train_size=100, test_size=None, random_state=42)
sets = []
for train_index, _ in shuffle_split.split(X_train):
    X_train_split = X_train[train_index]
    y_train_split = y_train[train_index]
    sets.append((X_train_split, y_train_split))


In [15]:
from sklearn.base import clone
from sklearn.metrics import accuracy_score
import numpy as np

forests = [clone(dicision_tree_best) for _ in range(1000)]

scores = []
for tree , (X_train_split, y_train_split) in zip(forests, sets):
    tree.fit(X_train_split, y_train_split)
    
    y_pred = tree.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

In [16]:
np.mean(scores)

0.8074535

In [17]:
Y_pred = np.empty([1000, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forests):
    Y_pred[tree_index] = tree.predict(X_test)

In [18]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [20]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.867